# How to save the processor form huggingface with pytorch?

In previous notebook I have save model to .pt. This time I wonder can I save the processor like the model.

## Save wav2vec2 processor 

In [1]:
from transformers import Wav2Vec2Processor

# Instantiating the processor
processor = Wav2Vec2Processor.from_pretrained(
    "airesearch/wav2vec2-large-xlsr-53-th")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
import torch

# Specify a path to save to
PATH = "./model/wav2vec2_processor.pt"

# Save
torch.save(processor, PATH)


## Load wav2vec2 processor 

### try method in [this pytorch site](https://pytorch.org/tutorials/recipes/recipes/save_load_across_devices.html)

In [3]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)


Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### try method another method


I think I work because 
1. I did not save using their method.
2. The pocessor did not have method state_dict().

In [4]:
model3 = torch.load(PATH)

In [5]:
model3

In [6]:
import audio2numpy as a2n
import numpy as np
import torchaudio

def load_mp3_to_array_fn(path: str):
    r"""replace: torchaudio.load(path) which cannot load .mp3"""
    speech_array, _ = a2n.audio_from_file(path)
    speech_array = torch.tensor(np.array([speech_array]))
    resampler = torchaudio.transforms.Resample(48_000, 16_000)
    tensor = resampler(speech_array).squeeze().numpy()
    return tensor


In [7]:
loaded_model = torch.jit.load(
    "./model/airesearch-wav2vec2-large-xlsr-53-th.pt")

In [8]:
# Test original model from huggingface
path = r"./mp3/common_voice_th_23646618.mp3"

tensor = load_mp3_to_array_fn(path)

inputs = model3(tensor, sampling_rate=16_000,
                   return_tensors="pt", padding=True)

input_values = inputs.input_values.to("cuda")

with torch.no_grad():
    logits = loaded_model(input_values)['logits']

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", model3.batch_decode(predicted_ids))


/home/somenoe/miniconda3/envs/py37/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811797118/work/torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)


Prediction: ['พวกเรา อาจจะ ต้องการ ความช่วยเหลือ จาก เธอ ในที่สุด']
